In [1]:
import pickle
import pandas as pd
import spacy

nlp = spacy.load('en')

import re
import pyLDAvis
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer 
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
STOPLIST = set(list(ENGLISH_STOP_WORDS) + ["n't","'s","'m","ca","'","'re","pron"])

In [2]:
# with open('../data/clean_tweets.pkl', 'rb') as picklefile:
#     df = pickle.load(picklefile)

In [ ]:
with open('../data/hashtag_corpus.pkl', 'rb') as picklefile:
    text = pickle.load(picklefile)

In [ ]:
text = df.sample(100).text.values

In [4]:
def get_tokens(text):  
    """Return lemmatized tokens."""
    text = re.sub(r"http\S+", "", text) 
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = ' '.join(letters_only.lower().split())
    try:
        tokens = [token.lemma_ for token in nlp(words)] 
    except:
        tokens = [token.lemma_ for token in nlp(words.decode('utf8'))] 
    filtered = [t for t in tokens if t != '' and t != ' ' and t != '\n' and t != '\n\n']
    return ' '.join(filtered)

In [ ]:
tokens = [get_tokens(tweet) for tweet in text]

In [ ]:
# def prep_pylda(docs, n_components = 5):   
#     vect = TfidfVectorizer(max_df = 0.5, max_features = 10000,
#                                  min_df = 5, stop_words = STOPLIST,
#                                  use_idf = True, tokenizer = None, ngram_range=(1, 3))
#     matrix = vect.fit_transform(docs)
#     vocab = vect.get_feature_names()
    
#     # fit transform lda
#     lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
#                 learning_method='online', learning_offset=50.,
#                 random_state=0, doc_topic_prior = .001)
#     doc_topic_dists = lda.fit_transform(matrix)
    
#     # prepare pyLDAvis stuff 
#     get_normed = lambda data: pd.DataFrame(data).div(data.sum(axis = 1), axis = 0)
#     doc_lengths = [len(tok) for tok in docs]
#     prepared = pyLDAvis.prepare(
#             doc_lengths = doc_lengths,
#             vocab = vocab,
#             term_frequency = np.asarray(matrix.sum(axis = 0)).ravel().tolist(),
#             topic_term_dists = get_normed(lda.components_),  
#             doc_topic_dists = get_normed(doc_topic_dists)) 
    
#     return prepared

In [ ]:
def prep_pylda(docs, n_components = 5):   
    vect = CountVectorizer(min_df=2)
    matrix = vect.fit_transform(docs)
    vocab = vect.get_feature_names()
    
    # fit transform lda
    lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    doc_topic_dists = lda.fit_transform(matrix)
    
    # prepare pyLDAvis stuff 
    get_normed = lambda data: pd.DataFrame(data).div(data.sum(axis = 1), axis = 0)
    doc_lengths = [len(tok) for tok in docs]
    prepared = pyLDAvis.prepare(
            doc_lengths = doc_lengths,
            vocab = vocab,
            term_frequency = np.asarray(matrix.sum(axis = 0)).ravel().tolist(),
            topic_term_dists = get_normed(lda.components_),  
            doc_topic_dists = get_normed(doc_topic_dists)) 
    
    return prepared

In [ ]:
prepared = prep_pylda(text, n_components = 20)

In [ ]:
# display in notebook
pyLDAvis.display(prepared)